Данный код приведен для примера отправки запросов к личному кабинету suppliers-api.wildberries.ru
Запрос работают только с версией сервера v1. Примеры запросов на v2 будут позже

Все запросы и описания взяты c файла https://images.wbstatic.net/portal/education/Kak_rabotat'_s_servisom_statistiki.pdf?abc=1612952230000

In [ ]:
import requests
import pandas as pd
import datetime as dt
import time
import matplotlib.pyplot as plt

#from itables import init_notebook_mode
#init_notebook_mode(all_interactive=False)

In [ ]:
#считываем ключи для v1. Ключи генерируются на сайте в личном кабинете
with open(r'.\keys\v1\x64.txt', 'r') as file:
    API_link_x64 = file.read().replace('\n', '')

DateFrom='2022-04-15' #в формате RFC3339
API_ADDRESS='https://suppliers-api.wildberries.ru'

In [ ]:
print(f'мой ключ {API_link_x64}')

# Функции

## requests GET

In [ ]:
def SendRequest(input_url,input_params):
    """отправляем запрос на wb
    input:
        input_url - web путь к обработчику wildberries.
        input_params - запрос jston к web api
    output:
        data - результат запроса
    """
    print('url',input_url)
    print('params',input_params)
        
    start_time = time.time()
    resp = requests.get(url=input_url, params=input_params)
    try:
        data = resp.json()
    except:
        print('Ошибка!', resp)
        return 0
    print("--- %s seconds ---" % (time.time() - start_time))
    if data!=None:
        print('got',len(data),'rows')
    else:
        print('No datas!!!')
    
    return data

In [ ]:
def GetInfo(input_quantity):
    """Получение информации по номенклатурам, их ценам, скидкам и промокодам. Если не указывать фильтры, вернётся весь товар.
    input:
        input_quantity - 2 - товар с нулевым остатком, 1 - товар с ненулевым остатком, 0 - товар с любым остатком
    output:
        result - результат запроса в формате df
    """
    print('Get stocks request...')
    url = f'{Api_address}/public/api/v1/info'
    params = {
        'quantity':input_quantity      
    } 
    result = SendRequest(url,params)
    if result==0: #если во время запроса была ошибка, то возвращаем 0
        return 0
    return pd.DataFrame(result,)

In [ ]:
def GetStocks(input_dateFrom):
    """отправляем запрос на информацию по складу
    !!!ВНИМАНИЕ!!!! Сервис статистики не хранит историю ОТСАТКОВ товаров, поэтому получить данные об остатках товаров на прошедшую, не сегодняшнюю, дату
    невозможно. Можно получить данные только на момент запроса к API
    !!!данные обновляются раз в сутки!!!
    input:
        input_dateFrom - дата и время от которого смотреть статистику. Формат RFC3339
    output:
        result - результат запроса в формате df
    """
    print('Get stocks request...')
    url = 'https://suppliers-stats.wildberries.ru/api/v1/supplier/stocks'
    params = {
        'dateFrom':input_dateFrom,
        'key':API_link_x64        
    } 
    result = SendRequest(url,params)
    if result==0: #если во время запроса была ошибка, то возвращаем 0
        return 0
    
    return pd.DataFrame(result)

In [ ]:
def GetOrders(input_dateFrom, is_flag):
    """отправляем запрос на список заказов
    !!!Данные обновляются 1 раз в 30 минут. (Точное время отображается в поле lastChangeDate)!!!
    !!!Гарантируется хранение данных продаж не более 90 дней от текущей даты.!!!
    input:
        input_dateFrom - дата и время от которого смотреть статистику Формат RFC3339
        is_flag -  0 - (или не указан в строке запроса), при вызове API возвращаются данные у которых значение поля lastChangeDate (дата время обновления информации в сервисе) больше переданного в вызов значения параметра dateFrom. При этом количество возвращенных строк данных варьируется в интервале от 0 до примерно 100000
                   1 - будет выгружена информация обо всех заказах или продажах с датой равной переданному параметру dateFrom (в данном случае время в дате значения не имеет). При этом количество возвращенных строк данных равно количеству всех заказов или продаж, сделанных в дате, переданной в параметре dateFrom. Ограничение на разовый запрос — около ста тысяч записей.
    output:
        result - результат запроса в формате df
    """
    print('Get orders request...')
    url = 'https://suppliers-stats.wildberries.ru/api/v1/supplier/orders'
    params = {
        'dateFrom':input_dateFrom,
        'flag':is_flag,
        'key':API_link_x64  
    } 
    result = SendRequest(url,params)
    if result==0: #если во время запроса была ошибка, то возвращаем 0
        return 0
    return pd.DataFrame(result)

In [ ]:
def GetSales(input_dateFrom, is_flag):
    """отправляем запрос на список продаж
    !!!Гарантируется хранение данных заказов не более 90 днейот текущей даты. !!!
    !!!Данные обновляются 1 раз в 30 минут. (Точное время отображается в поле lastChangeDate)!!!
    input:
        input_dateFrom - дата и время от которого смотреть статистику Формат RFC3339
        is_flag -  0 - (или не указан в строке запроса), при вызове API возвращаются данные у которых значение поля lastChangeDate (дата время обновления информации в сервисе) больше переданного в вызов значения параметра dateFrom. При этом количество возвращенных строк данных варьируется в интервале от 0 до примерно 100000.
                   1 - выгружена информация обо всех заказах или продажах с датой равной переданному параметру dateFrom (в данном случае время в дате значения  не имеет). При этом количество возвращенных строк данных равно количеству всех заказов или продаж, сделанных в дате, переданной в параметре dateFrom.
    output:
        result - результат запроса в формате df
    """
    print('Get sales request...')
    url = 'https://suppliers-stats.wildberries.ru/api/v1/supplier/sales'
    params = {
        'dateFrom':input_dateFrom,
        'flag':is_flag,
        'key':API_link_x64  
    } 
    result = SendRequest(url,params)
    if result==0: #если во время запроса была ошибка, то возвращаем 0
        return 0
    return pd.DataFrame(result)

In [ ]:
def GetReportDetailByPeriod(input_dateFrom, input_dateTo, input_limit, input_rrdid):
    """отправляем запрос на получение детального отчета
    По API можно получать данные за произвольный заданный период начиная с июня 2020г
    !!!В API хранятся данные не более трех последних месяцев.!!!
    input:
        input_dateFrom - дата и время от которого смотреть статистику Формат RFC3339
        input_dateTo - конечная дата периода.
        input_limit - максимальное количество строк отчета получаемых в результате вызова API. Рекомендуем загружать отчет небольшими частями, например, по 100 000 строк на один вызов
        input_rrdid – уникальный идентификатор строки отчета. Необходим для получения отчета частями. Загрузку отчета нужно начинать с rrdid = 0, и при последующих вызовах API передавать максимальное значение rrdid полученное в результате предыдущего вызова. Таким образом для загрузки одного отчета может понадобится вызывать API до тех пор, пока количество возвращаемых строк не станет равным нулю
    output:
        result - результат запроса в формате df
    """
    print('Get ReportDetailByPeriod request...')
    url = 'https://suppliers-stats.wildberries.ru/api/v1/supplier/reportDetailByPeriod'
    params = {
        'dateFrom':input_dateFrom,
        'dateTo':input_dateTo,
        'limit':input_limit,
        'rrdid':input_rrdid,
        'key':API_link_x64  
    } 
    result = SendRequest(url,params)
    if result==0: #если во время запроса была ошибка, то возвращаем 0
        return 0
    return pd.DataFrame(result)

In [ ]:
def GetExciseReport(input_dateFrom):
    """отправляем запрос на КиЗ
    input:
        input_dateFrom - дата и время от которого смотреть статистику Формат RFC3339
    output:
        result - результат запроса в формате df
    """
    print('Get kiz report request...')
    url = 'https://suppliers-stats.wildberries.ru/api/v1/supplier/excise-goods'
    params = {
        'dateFrom':input_dateFrom,
        'key':API_link_x64  
    } 
    result = SendRequest(url,params)
    if result==0: #если во время запроса была ошибка, то возвращаем 0
        return 0
    return pd.DataFrame(result)

## additation

In [ ]:
def Localization(input_df):
    #словарь для удобного чтения
    transl_col_dic={}
    transl_col_dic['date']='дата' # дата, когда данные появились в системе (по ним и отслеживатьизменение, появление новых)
    transl_col_dic['docnumber']='фискальный_номер'# номер фискального документа
    transl_col_dic['excise']= 'акциз'# акциз (он же киз)
    transl_col_dic['finishedPrice']= 'цена_товара_с_НДС' # цена товара с учетом НДС
    transl_col_dic['fiscalDt']= 'время_фискализации' #время фискализации
    transl_col_dic['fnnumber']= 'номер_фиск_накопителя' #номер фискального накопителя
    transl_col_dic['operationTypeId']= 'тип_операции' #тип операции (тип операции 1 - продажа, 2 - возврат)
    transl_col_dic['regNumber']= 'номер_ККТ' #регистрационный номер ККТ
    transl_col_dic['inn']='ИНН' #ИНН поставщика
    transl_col_dic['realizationreport_id']='Номер_отчета'
    transl_col_dic['suppliercontract_code']='Договор'
    transl_col_dic['rid']='id_заказа' #Уникальный идентификатор позиции заказа
    transl_col_dic['rr_dt']='Дата_операции'
    transl_col_dic['rrd_id']='Номер_строки'
    transl_col_dic['gi_id']='Номер_поставки'
    transl_col_dic['subject_name']='Предмет'
    transl_col_dic['NM_id']='Артикул'
    transl_col_dic['brand_name']='Бренд'
    transl_col_dic['sa_name']='Артикул_поставщика'
    transl_col_dic['ts_name']='Размер'
    transl_col_dic['barcode']='Баркод'
    transl_col_dic['doc_type_name']='Тип_документа'
    transl_col_dic['quantity']='Количество'
    transl_col_dic['retail_price']='Цена_розничная'
    transl_col_dic['retail_amount']='Сумма_продаж(Возвратов)'
    transl_col_dic['sale_percent']='Согласованная_скидка'
    transl_col_dic['commission_percent']='Процент_комиссии'
    transl_col_dic['office_name']='Склад'
    transl_col_dic['supplier_oper_name']='Обоснование_для_оплаты'
    transl_col_dic['order_dt']='Даты_заказа'
    transl_col_dic['sale_dt']='Дата_продажи'
    transl_col_dic['shk_id']='ШК'
    transl_col_dic['retail_price_withdisc_rub']='Цена_розничная_с_учетом'
    transl_col_dic['delivery_amount']='Колво_доставок'
    transl_col_dic['return_amount']='Колво_возвратов'
    transl_col_dic['delivery_rub']='Стоимость_логистики'
    transl_col_dic['gi_box_type_name']='Тип_коробов'
    transl_col_dic['product_discount_for_report']='Скидка_согласованная' #Согласованныйпродуктовыйдисконт
    transl_col_dic['supplier_promo']='Промокод'
    transl_col_dic['ppvz_spp_prc']='Скидка_покупателя' #СкидкапостоянногоПокупателя(СПП)'
    transl_col_dic['ppvz_kvw_prc_base']='Размер_кВВ_без_НДС'
    transl_col_dic['ppvz_kvw_prc']='Итоговый_кВВ_безНДС'
    transl_col_dic['ppvz_sales_commission']='Вознаграждение' #Вознаграждение с продаж до вычета услуг поверенного, без НДС
    transl_col_dic['ppvz_for_pay']='перечесление_за_реализацию' #К перечислению Продавцу за реализованный Товар
    transl_col_dic['ppvz_reward']='Возмещение' #Возмещение Расходов услуг поверенного
    transl_col_dic['ppvz_vw']='Вознаграждение_wb' #Вознаграждение Вайлдберриз (ВВ), без НДС
    transl_col_dic['ppvz_vw_nds']='Вознаграждение_wb_nds' #Вознаграждение Вайлдберриз (ВВ), с НДС
    transl_col_dic['ppvz_office_id']='Номер_офиса' #Номер_офиса доставки
    transl_col_dic['ppvz_office_name']='Наименование_офиса' #Наименование_офиса доставки
    transl_col_dic['ppvz_supplier_id']='Номер_партнера'
    transl_col_dic['ppvz_supplier_name']='Партнер'
    transl_col_dic['ppvz_inn']='ИНН_партнера'
    transl_col_dic['declaration_number']='Номер_томожн' #Номер таможенной декларации
    transl_col_dic['bonus_type_name']='Обоснование' #Обоснование штрафов и доплат

    #продажи
    transl_col_dic['gNumber']='номер_заказа'
    transl_col_dic['Number']='номер_документа'
    transl_col_dic['Date']='дата_продажи'
    transl_col_dic['lastChangeDate']='дата время обновления информации в сервисе'
    transl_col_dic['supplierArticle']='ваш артикул'
    transl_col_dic['techSize']='размер'
    transl_col_dic['barcode']='штрих-код'
    transl_col_dic['quantity']='кол-во'
    transl_col_dic['totalPrice']='розничная_цена'     #начальная_розничная_цена_товара  totalPrice*(100-discountPercent)/100
    transl_col_dic['discountPercent']='скидка' #согласованная скидка на товар
    transl_col_dic['isSupply']='договор_поставки'
    transl_col_dic['isRealization']='договор_реализации'
    transl_col_dic['promoCodeDiscount']='согласованный_промокод'
    transl_col_dic['warehouseName']='склад_отгрузки'
    transl_col_dic['countryName']='страна'
    transl_col_dic['oblastOkrugName']='округ'
    transl_col_dic['regionName']='регион'
    transl_col_dic['incomeID']='номер_поставки'
    transl_col_dic['saleID']='ID продажи' #'уникальный идентификатор продажи/возврата (SXXXXXXXXXX — продажа, RXXXXXXXXXX — возврат, DXXXXXXXXXXX — доплата, 'AXXXXXXXXX' – сторно продаж (все значения полей как у продажи, но поля с суммами и кол-вом с минусом как в возврате). SaleID='BXXXXXXXXX' - сторно возврата(все значения полей как у возврата, но поля с суммами и кол-вом с плюсом, в противоположность возврату)'
    transl_col_dic['Odid']='позиция_заказа' #уникальный идентификатор позиции заказа        
    transl_col_dic['spp']='скидка_покупателя' #согласованная скидка постоянного покупателя (СПП)'       
    transl_col_dic['forpay']='к_оплате'    #к перечислению поставщику     
    transl_col_dic['finished_price']='фактическая цена' #фактическая цена из заказа (с учетом всех скидок, включая и от ВБ)
    transl_col_dic['pricewithdisc']='цена_возаграждения' #цена, от которой считается вознаграждение поставщика forpay (с учетом всех согласованных скидок)) totalprice*((100 – discountPercent)/100 ) *((100 – promoCodeDiscount)/100 ) *((100 – spp)/100 )
    transl_col_dic['nmId']='код WB'
    transl_col_dic['subject']='предмет'
    transl_col_dic['category']='категория'
    transl_col_dic['brand']='бренд'
    
    #заказы
    transl_col_dic['gNumber']='номер_заказа' #объединяет все уникальные позиции одного заказа
    transl_col_dic['date']='дата_заказа'
    transl_col_dic['lastChangeDate']='дата_обновления' #Дата/времядатавремяобновленияинформациив'
    transl_col_dic['supplierArticle']='артикул'
    transl_col_dic['techSize']='размер'
    transl_col_dic['barcode']='штрих-код'
    transl_col_dic['quantity']='кол-во'
    transl_col_dic['totalPrice']='цена_со_скидкой' #ценадосогласованнойскидки/промо/спп
    transl_col_dic['discountPercent']='итоговая_скидка' #согласованныйитоговыйдисконт.
    transl_col_dic['warehouseName']='складотгрузки'
    transl_col_dic['oblast']='область'
    transl_col_dic['incomeID']='номер_поставки'
    transl_col_dic['odid']='уникальный_id'#'уникальныйидентификаторпозиции'
    transl_col_dic['nmid']='КодWB'
    transl_col_dic['subject']='предмет'
    transl_col_dic['category']='категория'
    transl_col_dic['brand']='бренд'
    transl_col_dic['is_cancel']='заказ_отменен'#'Отменазаказа.1–заказотменендо оплаты'

    #поставки
    transl_col_dic['lastChangeDate']='дата_обновления' # 'обновления информации сервисе
    transl_col_dic['supplierArticle']='артикул'
    transl_col_dic['techSize']='размер'
    transl_col_dic['Barcode']='штрих-код'
    transl_col_dic['Quantity']='кол-во' #"количество, доступное для продажи" —доступно на сайте, можно добавить в корзину
    transl_col_dic['isSupply']='договор_поставки'
    transl_col_dic['isRealization']='договор_реализации'
    transl_col_dic['quantityFull']='кол-вополное'
    transl_col_dic['quantityNotInOrders']='кол-во_не_в_заказе' #"количество не в заказе" — ислится на складе, и при этом не числится в незавершенном заказе
    transl_col_dic['warehouseName']='название_склада'
    transl_col_dic['inWayToClient']='впутикклиенту(штук)'
    transl_col_dic['inWayFromClient']='впутиотклиента(штук)'
    transl_col_dic['nmid']='код_WB'
    transl_col_dic['subject']='предмет'
    transl_col_dic['category']='категория'
    transl_col_dic['DaysOnSite']='кол-во_дней_на_сайте'
    transl_col_dic['brand']='бренд'
    transl_col_dic['SCCode']='код_контракта'

    
    transl_col_dic['incomeid']='номер_поставки'
    transl_col_dic['Number']='номер_УПД'
    transl_col_dic['Date']='дата_поступления'
    transl_col_dic['lastChangeDate']= 'дата_и_время_на_серивсе'#'датаивремяобновленияинформации сервисе'
    transl_col_dic['SupplierArticle']='артикул'
    transl_col_dic['TechSize']='размер'
    transl_col_dic['Barcode']='штрих-код'
    transl_col_dic['Quantity']='кол-во' 
    transl_col_dic['totalPrice']='цена_из_УПД'
    transl_col_dic['dateClose']='дата_принятия(закрытия)='
    transl_col_dic['warehouseName']='название_склада'
    transl_col_dic['nmid']='КодWB'
    transl_col_dic['status']='статус_поставки'

    input_df.rename(columns=transl_col_dic, inplace=True)
    
    return input_df

# Test

In [ ]:
#проверям,что все запросы отрабатывают корректно
dateTo= dt.datetime.now(date.timezone.utc) #для теста выберим сегодняшнюю дату
dateFrom= dateTo - dt.timedelta(days=1) #там, где необходим период, возьмем вчерашнюю дату
dateTo=dateTo.isoformat()
dateFrom=dateFrom.isoformat()

#это параметры обязательные для разных запросов и пример их вызова
limit=100
rrdid=0
flag=0
key=API_link_x64 
GetStocks(dateFrom)#отправляем запрос на информацию по складу
GetSales(dateFrom,flag)#отправляем запрос на список продаж
GetOrders(dateFrom,flag) #отправляем запрос на список заказов
GetReportDetailByPeriod(dateFrom,dateTo,limit,rrdid) #отправляем запрос на получение детального отчета
GetExciseReport(dateFrom) #отправляем запрос на КиЗ. Этот запрос почему то не работает

# main

Здесь можно уже играться с запросами и выводить отчеты с переводом полей или без

In [ ]:
dateFrom='2000-04-13' #берем произвольную дату
df_stocks=GetStocks(dateFrom) #отправляем на эту дату запрос по складу

In [ ]:
#выводим первые 5 строк
df_stocks[:5]

In [ ]:
translate_df=Localization(df_stocks) #если надо,переименоваем столбцы

In [ ]:
df_stocks[:5] #выводим уже с переименованием

In [ ]:
df_stocks.to_excel("stocks.xlsx") #сохраняем в excel,если надо в корневую директорию